# Generates and submits a qsub script to load and annotate edit BED files.
- Using annotator/0.0.14 (https://github.com/byee4/annotator)

In [1]:
import glob
import os
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info'

### Here, I want the joined BED files from 02 outputs

In [3]:
all_beds = glob.glob(os.path.join(input_dir, '*.bed'))
print(len(all_beds))
all_beds

37


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/sampled_500_RBFOX2_ORFS.fx.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/sampled_500_TIA1_ORFS.fx.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/sampled_500_RBFOX2_ORFS.barcodes_RBFOX2-TIA1_subset_more_noFeatureCells.fx.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/sampled_300_RBFOX2_ORFS.fx.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/sampled_200_RBFOX2_ORFS.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_groups/combined_outputs_w_cov_info/TIA1_O

In [4]:
def fix_chr(infile, outfile):
    with open(infile, 'r') as i:
        with open(outfile, 'w') as o:
            for line in i:
                o.write('chr{}'.format(line))
                
progress = tnrange(len(all_beds))
for bed in all_beds:
    if bed[-7:] == '.fx.bed':
        pass
    else:
        output_fx_bed = os.path.splitext(bed)[0] + ".fx.bed"
        fix_chr(bed, output_fx_bed)
    progress.update(1)

In [5]:
all_beds = glob.glob(os.path.join(input_dir, '*.fx.bed'))
len(all_beds)

19

In [6]:
gtfdb_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
cmds = []
for bed in all_beds:
    output_file = os.path.join(output_dir, os.path.basename(bed).replace('.fx.bed','.fx.annotated'))
    if not os.path.exists(output_file):
        cmd = 'module load annotator;annotator '
        cmd += '--output {} '.format(output_file)
        cmd += '--input {} '.format(bed)
        cmd += '--gtfdb {} '.format(gtfdb_file)
        cmd += '--species {} '.format('hg19')
        # cmd += '--transcript-priority-file {} '.format(priority)
        # cmd += '--gene-priority-file {}'.format(priority)
        cmds.append(cmd)
        
len(cmds)

2

In [7]:
Submitter(commands=cmds, job_name='annotate_editing_sites', array=True, nodes=1, ppn=2, submit=True, walltime='2:00:00')

Writing 2 tasks as an array-job.
Wrote commands to annotate_editing_sites.sh.
Submitted script to queue home.
 Job ID: 21389167
